In [ ]:
# Read processed_things.json

import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import folium
import os
import math

def get_coordinates_by_thing_name():
    with open(f'things.json', 'r', encoding='utf-8') as JSON:
        things = json.load(JSON)
    thing_locations = {}
    for thing in things:
        location = None
        try:
            location = thing["Locations"][0]["location"]["geometry"]["coordinates"][0][0]
        except:
            location_found = False
            # Try to extract location from observed area from datastreams
            for datastream in thing["Datastreams"]:
                if "observedArea" not in datastream:
                    continue
                if not isinstance(datastream["observedArea"]["coordinates"][0], list):
                    location = datastream["observedArea"]["coordinates"]
                else:
                    location = datastream["observedArea"]["coordinates"][0]
                location_found = True
                break
            if not location_found:
                with open(f"last_thing_with_no_location_data.json", 'w') as fp:
                    json.dump(thing, fp)
                print(f"Location not found for thing {thing['name']}. Look at last_thing_with_no_location_data.json for more information.")
        if location is None:
            continue
        if isinstance(location[0], list):
            location = location[0]
        thing_locations[thing["name"]] = location
    return thing_locations

def get_median_for_metric_of_thing(dayAndHourList):
    values = []
    for day in dayAndHourList:
        for hour in day:
            if hour != -1.0:
                values.append(hour)
            
    median = np.median(np.array(values))
    return median

with open('processed_things.json') as f:
    processed_things = json.load(f)
    
thing_locations = get_coordinates_by_thing_name()

maps_data = [
    {
        "key": "PrimarySignalMissingCount",
        "max": 0,
    },
    {
        "key": "CycleSecondMissingCount",
        "max": 0,
    },
    {
        "key": "TotalSkippedCycles",
        "max": 0,
    },
    {
        "key": "TotalRemovedCycleCount",
        "max": 0,
    },
    {
        "key": "TotalInvalidCycleLengthCount",
        "max": 0,
    },
    {
        "key": "TotalInvalidCycleTransitionCount",
        "max": 0,
    },
    {
        "key": "TotalInvalidCycleMissingCount",
        "max": 0,
    },
    {
        "key": "TotalCyclesCount",
        "max": 0,
    },
    {
        "key": "MedianMetric",
        "max": 0,
    },
]
    
for thing_name, thing in processed_things.items():
    for map_data in maps_data:
        if map_data["key"] == "MedianMetric":
            median = get_median_for_metric_of_thing(thing['Metrics'])
            if median > map_data["max"]:
                map_data["max"] = median
        elif thing[map_data["key"]] > map_data["max"]:
            map_data["max"] = thing[map_data["key"]]
    
for map_data in maps_data:
    figure = folium.Figure(width=600, height=500)
    map = folium.Map(location=[53.57532, 10.01534], zoom_start=10).add_to(figure)
    folium.TileLayer('cartodbpositron').add_to(map)
    for thing_name, thing in processed_things.items():
        if map_data["key"] == "MedianMetric":
            median = get_median_for_metric_of_thing(thing['Metrics'])
            percentage = median / map_data["max"]
            if math.isnan(percentage):
                color = '#%02x%02x%02x' % (0, 0, 0)
            else:
                color = '#%02x%02x%02x' % (int(255 * percentage), int(255 * (1 - percentage)), 0)
        else:
            percentage = thing[map_data["key"]] / map_data["max"] if map_data["max"] != 0 else 0
            color = '#%02x%02x%02x' % (int(255 * percentage), int(255 * (1 - percentage)), 0)
        
        try:
            coordinates = [thing_locations[thing_name][1],thing_locations[thing_name][0]]
        except:
            default_coordinates = [53.683899,9.867225]
            # randomize default coordinates
            default_coordinates[0] += np.random.uniform(-0.0001, 0.0001)
            default_coordinates[1] += np.random.uniform(-0.0001, 0.0001)
            coordinates = [default_coordinates[1],default_coordinates[0]]
            
        map_element = folium.CircleMarker(coordinates, radius=3, color=color)
        
        popup_text = f"""
        <b>Thing: {thing_name}</b><br>
        Primary signal missing count: {thing['PrimarySignalMissingCount']}<br>
        Cycle second missing count: {thing['CycleSecondMissingCount']}<br>
        Skipped cycles: {thing['TotalSkippedCycles']}<br>
        Removed cycles: {thing['TotalRemovedCycleCount']}<br>
        Invalid cycle length count: {thing['TotalInvalidCycleLengthCount']}<br>
        Invalid cycle transition count: {thing['TotalInvalidCycleTransitionCount']}<br>
        Invalid cycle missing count: {thing['TotalInvalidCycleMissingCount']}<br>
        Total cycles: {thing['TotalCyclesCount']}<br>
        Median metric: {median}<br>
        """
        folium.Popup(popup_text, max_width=300,min_width=300).add_to(map_element)
        map_element.add_to(map)
    title_html = '''
        <h3 align="center" style="font-size:16px"><b>{}</b></h3>
            '''.format(map_data["key"])
    map.get_root().html.add_child(folium.Element(title_html))
    display(map)

    